In [1]:
from typing import List, Dict, Tuple
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
dev = 'cpu'
if torch.cuda.is_available():
    dev = 'cuda:0'
print("Running on:",dev)
device = torch.device(dev)

Running on: cpu


# Quivers and quiver representations

The quiver class is designed to define an acyclic quiver with no double edges. The initial quiver is not required to have a bias vertex; such a vertex can be added with the ```add_bias()``` method.

The input list of vertices is meant to be a list of strings, one for each vertex. It is best to avoid the label ```bias``` among the vertices. 

The input list of edges is meant to be a tuple ```e = (e[0], e[1])``` where ```e[0]``` is the source and ```e[1]``` is the target. No double edges or loops are allowed.

In [2]:
class quiver:
    """Quiver class. Vertices are given as a list of strings. Edges are given as a list of pairs.
    
    Attributes and Methods.
    """
    def __init__(self, vertices : List[str], edges : List[Tuple[str]]):
        self.vertices = vertices
        # Add assert to check no repeated vertices
        # E.g. assert len(set(vertices)) == len(vertices)
        # Assert there is no bias initially, or if there is, it has the desired properties
        
        self.edges = edges
        # Add assert to check that edges is a list of pairs
        # First entry of the pair is the sourse, second is the target
        # Source and target of each edge should be in the vertex set      
        # Separate class for edges? vertices?
        
        # Get the sources and sinks
        sources = set(self.vertices)
        sinks = set(self.vertices)
        for e in self.edges:
            sources.discard(e[1])
            sinks.discard(e[0])
        self.sources = sources
        self.sinks = sinks
    
    # Check that the quiver is acyclic
    def check_acyclic(self):
        None
        # One way: find all sources, do depth-first search
        
    # Check that the vertices are in topological order    
    def check_top_order(self):
        indices = {}
        for i,v in enumerate(self.vertices):
            indices[v] = i
        return all([indices[e[0]] <  indices[e[1]] for e in self.edges])
        
    # Get the incoming edges for every vertex
    def get_incoming(self, vertex):
        assert vertex in self.vertices, "No such vertex found"
        return [e for e in self.edges if e[1] == vertex]
        # Can get the incoming neighbors as [e[1] for e in self.get_incoming(vertex)]
        
    # Add a bias vertex. Considering alternatives to this ... 
    def add_bias(self):
        # Add bias vertex. This will not disturb the topological order.
        for v in self.vertices:
            if v not in self.sources:
                self.edges.append(('bias', v))
        self.vertices = ['bias'] + self.vertices
        return
    
    ######
    # Check if a vertex is a sink (Don't really need this any more)
    def is_sink(self, vertex):
        assert vertex in self.vertices, "No such vertex found"
        return all([e[0] != vertex for e in self.edges])
    


In [3]:
class quiver_rep:
    """Quiver representation class. Input a quiver with dimension vector and a matrix for each edge."""
    def __init__(self, quiver: quiver, dims: Dict[str,int], matrices: Dict[str, np.array]):
        self.quiver = quiver
        self.dims = dims
        self.matrices = matrices
        
        # Check the dimension vector
        assert len(dims) == len(self.quiver.vertices), "Inappropriate dimension vector"
        for v in dims:
            assert v in self.quiver.vertices, "Inappropriate dim vector"
            assert isinstance(dims[v], int) and dims[v] >=0, "Dimension needs to be a positive integer"
        assert dims['bias'] == 1, "Dimension at bias needs to be 1"

            
        # Check the matrices
        assert len(matrices) == len(self.quiver.edges), "Matrices error"
        for e in matrices:
            assert e in self.quiver.edges, "Matrices error"
            assert isinstance(matrices[e], np.ndarray), "Matrices error" # May need fixing
            assert np.shape(matrices[e]) == (dims[e[1]], dims[e[0]]), "Dimension error"
            
            
    # Compute the reduced dimension vector
    def comp_dims_red(self) -> Dict:
        
        assert self.quiver.check_top_order(), "Order of the vertices is not topological"

        dims_red = {}
        for i in self.quiver.vertices:
            if i == 'bias' or i in self.quiver.sources or i in self.quiver.sinks:
                dims_red[i] = self.dims[i]
            else:
                incoming = self.quiver.get_incoming(i)
                dims_red[i] = min(self.dims[i], sum([dims_red[e[0]] for e in incoming]) )

                    
        self.dims_red = dims_red        
        return dims_red
    
    
    # Auxiliary function
    def padzeros(self, M, newrows, newcols = None):
        oldrows, oldcols = M.shape
        if newcols == None:
            newcols = oldcols
        return np.pad(M,((0,newrows-oldrows),(0,newcols-oldcols)),mode="constant")
 
    
    # QR dimensional reduction algorithm
    def QRDimRed(self, verbose : bool = False ):
        dims = self.dims
        matrices = self.matrices
        quiver = self.quiver
        vertices = quiver.vertices
        edges = quiver.edges

        # Check that vertices are in a topological order
        assert quiver.check_top_order(), "Order of the vertices is not topological"

        # Compute the reduced dimension vector
        dims_red = self.comp_dims_red()
        # print(dims, dims_red)

        # Q = dictionary mapping each vertex to an orthogonal matrix
        Q = {}

        # Vmatrices = matrices of the reduced representation V, mapping each edge to a matrix
        Vmatrices = {}

        if verbose:
            print(quiver.edges)
            print(quiver.vertices)

        for i in vertices:
            incoming = quiver.get_incoming(i)

            # Case of a source vertex
            if incoming == []:
                Q[i] = np.eye(dims[i])

            # Case of a hidden vertex    
            elif i not in quiver.sinks:

                # Compute the matrix to be QR-decomposed
                M = np.array([])
                for e in incoming:
                    # Transform weights on incoming edges
                    Qj = Q[e[0]]
                    Me = matrices[e] @ Qj[:,:dims_red[e[0]]]
                    if np.shape(M) == (0,):
                        M = Me
                    else:
                        M = np.hstack((M, Me))

                Q[i], R = np.linalg.qr(M, mode="complete")

                # Case of reduction 
                if dims_red[i] < dims[i]: 
                    R = R[:dims_red[i]]

                # Process and add to the dictionaries
                for e in incoming:                       
                    # Extract V_e from R_i for all incoming edges e
                    Vmatrices[e] = R[:,:dims_red[e[0]]]
                    R = R[:,dims_red[e[0]]:]

            # Case of a sink (no reduction)
            else:
                Q[i] = np.eye(dims[i])
                for e in incoming:
                    # Transform weights on incoming edges
                    Qj = Q[e[0]]
                    Vmatrices[e] = matrices[e] @ Qj[:,:dims_red[e[0]]]                


        # Make V into a representation
        V = quiver_rep(quiver, dims_red, Vmatrices)

        # Verify that V is a subrepresentation of Q^{-1} W  
        for e in quiver.edges:
            Qi = Q[e[0]]
            Qj = Q[e[1]]
            max_diff = np.max(np.abs(np.transpose(Qj) @ matrices[e] @ Qi[:,:dims_red[e[0]]] 
                         - self.padzeros(Vmatrices[e], dims[e[1]])))
            assert max_diff < 1e-10, "Error in the algorithm"

        return Q, V
    
    
    def reduced_representation(self, verbose : bool = False ):
        return self.QRDimRed(verbose)[1]
                
    def transformed_representation(self):
        Q, V = self.QRDimRed()
        transformed_mat_dict = {}
        for e in self.quiver.edges:
            transformed_mat_dict[e] = np.transpose(Q[e[1]]) @ self.matrices[e] @ Q[e[0]]
        
        Q_inv_W = quiver_rep(self.quiver, self.dims, transformed_mat_dict)
        
        return Q_inv_W

### Linear Feedforward Function

In [ ]:
# Linear feedforward function, ignoring biases
# Might turn out not to be super necessary

def lin_ff(W : quiver_rep) -> np.array:
    dims = W.dims
    matrices = W.matrices
    quiver = W.quiver
    vertices = quiver.vertices
    edges = quiver.edges
    
    assert quiver.check_top_order(), "Order of the vertices is not topological"
    
    # Dictionary for partial feedforward functions:
    partial = {}
    input_dim = 0
    # input_dim = sum( [dims[i] for i in quiver.sources])
    
    for i in quiver.vertices:
        incoming = quiver.get_incoming(i)
        
        # Case of a source
        if i in quiver.sources:
            
            # Update matrices already defined
            for j in partial:
                partial[j] = np.hstack((partial[j], np.zeros((dims[j],dims[i]))))
                
            # Define the new matrix as a projection matrix
            if input_dim == 0:
                partial[i] = np.eye(dims[i])
            else:
                partial[i] = np.hstack((np.zeros((dims[i], input_dim)), np.eye(dims[i])))
            input_dim += dims[i]

        # Case of a hidden vertex or a sink
        else:
            
            # Compute the matrix to be added
            A = np.zeros((dims[i], input_dim))
            for e in incoming:
                # Ignore biases for now
                if e[0] != 'bias':
                    A += matrices[e] @ partial[e[0]]
            partial[i] = A
         
    # Compute the final output matrix by stacking the matrices for sinks
    result = np.array([])
    for i in quiver.sinks:
        if np.shape(result) == (0,):
            result = partial[i]
        else:
            result = np.vstack((result, partial[i]))
                        
    return result

# Dimensional reduction algorithm

In [178]:
# Auxiliary function

def padzeros(M,newrows,newcols = None):
    oldrows, oldcols = M.shape
    if newcols == None:
        newcols = oldcols
    return np.pad(M,((0,newrows-oldrows),(0,newcols-oldcols)),mode="constant")

In [179]:
# QR dimensional reduction algorithm
# This is incorporated in the quiver_rep class

def QRDimRed(W : quiver_rep, verbose : bool = False ):
    dims = W.dims
    matrices = W.matrices
    quiver = W.quiver
    vertices = quiver.vertices
    edges = quiver.edges
    
    # Check that vertices are in a topological order
    assert quiver.check_top_order(), "Order of the vertices is not topological"
    
    # Compute the reduced dimension vector
    dims_red = W.comp_dims_red()
    # print(dims, dims_red)
    
    # Q = dictionary mapping each vertex to an orthogonal matrix
    Q = {}
    
    # Vmatrices = matrices of the reduced representation V, mapping each edge to a matrix
    Vmatrices = {}
    
    if verbose:
        print(quiver.edges)
        print(quiver.vertices)
    
    for i in vertices:
        incoming = quiver.get_incoming(i)
        
        # Case of a source vertex
        if incoming == []:
            Q[i] = np.eye(dims[i])
            
        # Case of a hidden vertex    
        elif i not in quiver.sinks:
            
            # Compute the matrix to be QR-decomposed
            M = np.array([])
            for e in incoming:
                # Transform weights on incoming edges
                Qj = Q[e[0]]
                Me = matrices[e] @ Qj[:,:dims_red[e[0]]]
                if np.shape(M) == (0,):
                    M = Me
                else:
                    M = np.hstack((M, Me))
                    
            Q[i], R = np.linalg.qr(M, mode="complete")

            # Case of reduction 
            if dims_red[i] < dims[i]: 
                R = R[:dims_red[i]]
                
            # Process and add to the dictionaries
            for e in incoming:                       
                # Extract V_e from R_i for all incoming edges e
                Vmatrices[e] = R[:,:dims_red[e[0]]]
                R = R[:,dims_red[e[0]]:]
                
        # Case of a sink (no reduction)
        else:
            Q[i] = np.eye(dims[i])
            for e in incoming:
                # Transform weights on incoming edges
                Qj = Q[e[0]]
                Vmatrices[e] = matrices[e] @ Qj[:,:dims_red[e[0]]]                

    
    # Make V into a representation
    V = quiver_rep(quiver, dims_red, Vmatrices)

    # Verify that V is a subrepresentation of Q^{-1} W  
    for e in quiver.edges:
        Qi = Q[e[0]]
        Qj = Q[e[1]]
        max_diff = np.max(np.abs(np.transpose(Qj) @ matrices[e] @ Qi[:,:dims_red[e[0]]] 
                     - padzeros(Vmatrices[e], dims[e[1]])))
        assert max_diff < 1e-10, "Error in the algorithm"

    return Q, V

# Example 1

In [155]:
# EXAMPLE
# Quiver with skip connections and no bias

vertex_list = ['a', 'b', 'c', 'd']
edge_list = [('a', 'b'), ('a','c'), ('b','c'), ('c', 'd')]

quiv_ex = quiver(vertex_list, edge_list)
# print(Q.vertices, Q.edges)

# Test the methods
print(quiv_ex.get_incoming('a'), quiv_ex.get_incoming('b'), quiv_ex.get_incoming('c'), quiv_ex.get_incoming('d'))
print(quiv_ex.is_sink('a'), quiv_ex.is_sink('b'), quiv_ex.is_sink('c'), quiv_ex.is_sink('d'))
print(quiv_ex.check_top_order())
print(quiv_ex.sources)
quiv_ex.add_bias()
print(quiv_ex.vertices)
print(quiv_ex.edges)

[] [('a', 'b')] [('a', 'c'), ('b', 'c')] [('c', 'd')]
False False False True
True
{'a'}
['bias', 'a', 'b', 'c', 'd']
[('a', 'b'), ('a', 'c'), ('b', 'c'), ('c', 'd'), ('bias', 'b'), ('bias', 'c'), ('bias', 'd')]


In [10]:
# Representation of this quiver

dim_vector = {'bias' : 1, 'a': 2, 'b': 4, 'c': 8, 'd': 2 }

maps = {('a', 'b') : np.random.rand(4, 2), 
        ('a', 'c') : np.random.rand(8, 2), 
        ('b', 'c') : np.random.rand(8, 4),
        ('c', 'd') : np.random.rand(2, 8),
        ('bias', 'b') : np.random.rand(4, 1),
        ('bias', 'c') : np.random.rand(8, 1),
        ('bias', 'd') : np.random.rand(2, 1)}

ex_rep = quiver_rep(quiv_ex, dim_vector, maps)
print(ex_rep.comp_dims_red())

{'bias': 1, 'a': 2, 'b': 3, 'c': 6, 'd': 2}


In [11]:
# Check the algorithm

Q_ex ,V_ex = QRDimRed(ex_rep)
print(ex_rep.comp_dims_red())
print([np.shape(Q_ex[i]) for i in Q_ex])
print([np.shape(V_ex.matrices[e]) for e in V_ex.matrices])

{'bias': 1, 'a': 2, 'b': 3, 'c': 6, 'd': 2}
[(1, 1), (2, 2), (4, 4), (8, 8), (2, 2)]
[(3, 2), (3, 1), (6, 2), (6, 3), (6, 1), (2, 6), (2, 1)]


In [12]:
lin_ff(ex_rep)

array([[7.84656088, 7.27676386],
       [6.16817118, 5.87194995]])

In [13]:
ex_rep.matrices[('c','d')] @ ( ex_rep.matrices[('a','c')] + ex_rep.matrices[('b','c')] @ ex_rep.matrices[('a','b')] )

array([[7.84656088, 7.27676386],
       [6.16817118, 5.87194995]])

In [14]:
lin_ff(V_ex)

array([[7.84656088, 7.27676386],
       [6.16817118, 5.87194995]])

# Example 2

In [4]:
# EXAMPLE
# Quiver with multiple inputs and outputs

vertex_list2 = ['a', 'b', 'c', 'd', 'e']
edge_list2 = [('a', 'c'), ('b','c'), ('c','d'), ('c', 'e')]

quiv_ex2 = quiver(vertex_list2, edge_list2)
# print(Q.vertices, Q.edges)

# Test the methods
print(quiv_ex2.get_incoming('a'), quiv_ex2.get_incoming('b'), quiv_ex2.get_incoming('c'), 
      quiv_ex2.get_incoming('d'), quiv_ex2.get_incoming('e'))
print(quiv_ex2.is_sink('a'), quiv_ex2.is_sink('b'), quiv_ex2.is_sink('c'), 
      quiv_ex2.is_sink('d'), quiv_ex2.is_sink('e'))
print(quiv_ex2.check_top_order())
print(quiv_ex2.sources)
quiv_ex2.add_bias()
print(quiv_ex2.vertices)
print(quiv_ex2.edges)

[] [] [('a', 'c'), ('b', 'c')] [('c', 'd')] [('c', 'e')]
False False False True True
True
{'b', 'a'}
['bias', 'a', 'b', 'c', 'd', 'e']
[('a', 'c'), ('b', 'c'), ('c', 'd'), ('c', 'e'), ('bias', 'c'), ('bias', 'd'), ('bias', 'e')]


In [7]:
# Representation of this quiver

dim_vector2 = {'bias' : 1, 'a': 1, 'b': 2, 'c': 8, 'd': 2 , 'e': 6}

maps2 = {('a', 'c') : np.random.rand(8, 1), 
        ('b', 'c') : np.random.rand(8, 2), 
        ('c', 'd') : np.random.rand(2, 8),
        ('c', 'e') : np.random.rand(6, 8),
        ('bias', 'c') : np.random.rand(8, 1),
        ('bias', 'd') : np.random.rand(2, 1),
        ('bias', 'e') : np.random.rand(6, 1)}

ex_rep2 = quiver_rep(quiv_ex2, dim_vector2, maps2)
print(ex_rep2.comp_dims_red())

{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}


In [8]:
# Check the algorithm

Q_ex2 ,V_ex2 = QRDimRed(ex_rep2)
print(ex_rep2.comp_dims_red())
print([np.shape(Q_ex2[i]) for i in Q_ex2])
print([np.shape(V_ex2.matrices[e]) for e in V_ex2.matrices])

NameError: name 'QRDimRed' is not defined

In [9]:
lin_ff(ex_rep2)

NameError: name 'lin_ff' is not defined

In [10]:
lin_ff(V_ex2)

NameError: name 'lin_ff' is not defined

# Quiver Neural Networks

In [167]:
class RadAct(nn.Module):
    def __init__(self, eta = F.relu):
        super().__init__()
        self.eta = eta
        self.shift = 0 
        # Add internal bias/shift later
        
    def forward(self,x):
        # x: [Batch x Channel]
        # Radial activations
        r = torch.linalg.norm(x, dim=-1) 
        if torch.min(r) < 1e-6:
            r += 1e-6
        scalar = self.eta(r + self.shift) / r
        return x * scalar.unsqueeze(-1)   

In [188]:
class QuiverNN(nn.Module):
    
    def __init__(self, eta: float , quiver: quiver, dims: Dict[str,int] ):
        super().__init__()
        self.eta = eta
        self.quiver = quiver
        self.dims = dims
        self.matrices = nn.ModuleDict()
        for e in quiver.edges:
            self.matrices[self.edge_tup_to_str(e)] = nn.Linear(self.dims[e[0]], self.dims[e[1]], bias = False)
        
        assert quiver.check_top_order(), "Vertices not in topological order."
        
        self.act = RadAct(self.eta)
        #self.quiver_rep = quiver_rep()
        
        # Assert statement to check that dims is a dimension vector for Q
        
        # Reduced dimension vector
    
    def edge_tup_to_str(self,e):
        (t,h) = e
        return t + " to " + h
    
    def get_matrix(self,e):
        return self.matrices[self.edge_tup_to_str(e)].weight

    def edge_str_to_tup(self,e):
        t,to,h = e.split()
        return (t,h)

    def forward(self, x, non_linear = True):
        
        #Initialize Data Flow
        h = {}
        h['bias'] = torch.tensor(1.0)
        for v in self.quiver.sources:
            h[v] = x[v]
            batch_size = x[v].shape[0]
        
        #Assert batchsize is same for all sources 
        
        for v in self.quiver.vertices:
            if v not in self.quiver.sources:
                h[v] = torch.zeros(batch_size,self.dims[v])
                for e in self.quiver.get_incoming(v):
                    e_lin = self.matrices[self.edge_tup_to_str(e)]
                    h[e[1]] += e_lin(h[e[0]])
                if non_linear:
                    h[v] = self.act(h[v])
        
        out = {}
        for v in self.quiver.sinks:
            out[v] = h[v]
            
        return out
            
    
    def set_weights(self, new_weights: quiver_rep):
        assert new_weights.quiver == self.quiver, "weights have different quiver"
        
        for e in self.quiver.edges:
            self.matrices[self.edge_tup_to_str(e)].weight = \
                torch.nn.Parameter(torch.tensor(new_weights.matrices[e],dtype = torch.float))
        self.dims = new_weights.dims
    
    def set_activation_biases(self, new_biases: List[float]):    
        None

    def export_weights(self) -> quiver_rep:
        quiver_rep_matrix_dict = {}
        for e in self.quiver.edges:
            quiver_rep_matrix_dict[e] = self.matrices[self.edge_tup_to_str(e)].weight.detach().cpu().numpy()
        return quiver_rep(self.quiver, self.dims, quiver_rep_matrix_dict)
    
    def export_activation_biases(self) -> List[float]:
        None
    
    def export_reduced_weights(self) -> quiver_rep:
        exported_rep = self.export_weights()
        return exported_rep.reduced_representation()
    
    def transformed_network(self):
        exported_rep = self.export_weights()
        rep_transformed = exported_rep.transformed_representation()
        net_trans = QuiverNN(self.eta, self.quiver, self.dims)
        net_trans.set_weights(rep_transformed)
        #net_trans.set_activation_biases(self.export_activation_biases())
        return net_trans
        
    def reduced_network(self):
        reduced_rep = self.export_reduced_weights()
        net_reduced = QuiverNN(self.eta, self.quiver, reduced_rep.dims)
        net_reduced.set_weights(reduced_rep)
        #net_trans.set_activation_biases(self.export_activation_biases())
        return net_reduced

## Example

In [189]:
vertex_list2 = ['a', 'b', 'c', 'd', 'e']
edge_list2 = [('a', 'c'), ('b','c'), ('c','d'), ('c', 'e')]
dim_vector2 = {'bias' : 1, 'a': 1, 'b': 2, 'c': 8, 'd': 2 , 'e': 6}
quiv_ex2 = quiver(vertex_list2, edge_list2)
quiv_ex2.add_bias()

quiverNN = QuiverNN(eta = F.relu, quiver = quiv_ex2, dims = dim_vector2)

# Representation of this quiver

maps2 = {('a', 'c') : np.random.rand(8, 1), 
        ('b', 'c') : np.random.rand(8, 2), 
        ('c', 'd') : np.random.rand(2, 8),
        ('c', 'e') : np.random.rand(6, 8),
        ('bias', 'c') : np.random.rand(8, 1),
        ('bias', 'd') : np.random.rand(2, 1),
        ('bias', 'e') : np.random.rand(6, 1)}

ex_rep2 = quiver_rep(quiv_ex2, dim_vector2, maps2)
print(ex_rep2.comp_dims_red())

quiverNN.set_weights(ex_rep2)

{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}


In [190]:
x = {'a': torch.tensor([[1.0]]),'b': torch.tensor([[1.0,1.0]])}
quiverNN(x)

{'d': tensor([[4.4020, 6.7395]], grad_fn=<MulBackward0>),
 'e': tensor([[7.0381, 7.4217, 6.7195, 3.8931, 5.6222, 5.6682]],
        grad_fn=<MulBackward0>)}

In [191]:
quiverNN.export_weights()

In [192]:
quiverNN.export_reduced_weights()

In [193]:
quiverNN.export_weights().comp_dims_red()

{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}

In [194]:
red_quiverNN = quiverNN.reduced_network()
red_quiverNN

QuiverNN(
  (matrices): ModuleDict(
    (a to c): Linear(in_features=1, out_features=4, bias=False)
    (b to c): Linear(in_features=2, out_features=4, bias=False)
    (c to d): Linear(in_features=4, out_features=2, bias=False)
    (c to e): Linear(in_features=4, out_features=6, bias=False)
    (bias to c): Linear(in_features=1, out_features=4, bias=False)
    (bias to d): Linear(in_features=1, out_features=2, bias=False)
    (bias to e): Linear(in_features=1, out_features=6, bias=False)
  )
  (act): RadAct()
)

In [195]:
red_quiverNN = quiverNN.reduced_network()
x = {'a': torch.tensor([[1.0]]),'b': torch.tensor([[1.0,1.0]])}
red_quiverNN(x)

{'d': tensor([[4.4020, 6.7395]], grad_fn=<MulBackward0>),
 'e': tensor([[7.0381, 7.4217, 6.7195, 3.8931, 5.6222, 5.6682]],
        grad_fn=<MulBackward0>)}

In [196]:
print(quiverNN.dims)
print(quiverNN.export_reduced_weights().dims)
for e in T.quiver.edges:
    print()
    print(e)
    print(T.get_matrix(e).detach().cpu().numpy().round(2))

{'bias': 1, 'a': 1, 'b': 2, 'c': 8, 'd': 2, 'e': 6}
{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}

('a', 'c')
[[-2.03]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [ 0.  ]
 [-0.  ]
 [-0.  ]
 [ 0.  ]]

('b', 'c')
[[-1.77 -2.22]
 [ 1.19  0.47]
 [-0.    0.96]
 [ 0.    0.  ]
 [-0.   -0.  ]
 [ 0.    0.  ]
 [-0.    0.  ]
 [-0.    0.  ]]

('c', 'd')
[[-1.23  0.88 -2.15 -1.43  0.34 -0.4  -0.11  0.39]
 [-1.17  0.63 -2.67 -1.68  0.5  -0.12  0.15  0.3 ]]

('c', 'e')
[[ 0.93 -0.04  1.12  0.56 -0.65 -0.09  0.68 -0.06]
 [ 0.99  0.35  1.09  0.55  0.07 -0.28  0.61 -0.32]
 [ 0.76 -0.31  0.36  0.34 -0.77 -0.45  0.3  -0.8 ]
 [ 0.99  0.3   1.13  0.79 -0.64 -0.46  0.29 -0.45]
 [ 0.92 -0.49  0.96  0.65 -1.02 -0.84  0.23 -0.57]
 [ 1.19  0.02  1.4   0.35 -0.54 -0.64  0.14 -0.63]]

('bias', 'c')
[[-1.4 ]
 [ 1.02]
 [-0.49]
 [-0.9 ]
 [ 0.  ]
 [ 0.  ]
 [-0.  ]
 [ 0.  ]]

('bias', 'd')
[[0.79]
 [0.07]]

('bias', 'e')
[[0.7 ]
 [0.72]
 [0.69]
 [0.8 ]
 [0.02]
 [0.72]]


## Test Theorem

In [197]:
W = quiverNN.export_weights()
T = quiverNN.transformed_network()
Q,V = quiverNN.export_weights().QRDimRed()

In [ ]:
def single_mask(red_rows, red_cols, orig_rows, orig_cols):
    assert red_rows <= orig_rows 
    assert red_cols <= orig_cols
    mask = torch.ones((orig_rows, orig_cols))
    for j in range(red_rows, orig_rows):
        mask[j][:red_cols] = 0
    return mask

In [209]:
def gamma(model, x, lr):
    
    y = model(x)
    
    L = torch.tensor(0.0)
    for k in y:
        L += y[k].pow(2).mean()
    L.backward()
    
    with torch.no_grad(): 
        for e in model.quiver.edges:
            p = model.get_matrix(e)
            p -= lr * p.grad
    
    print(L)
    
    return model
    


In [222]:
def gammaPGD(model, x, lr):
    
    y = model(x)
    
    dims = model.dims 
    red_dims = model.export_reduced_weights().dims
    
    L = torch.tensor(0.0)
    for k in y:
        L += y[k].pow(2).mean()
    L.backward()
    
    with torch.no_grad(): 
        for e in model.quiver.edges:
            p = model.get_matrix(e)
            mask_grad = single_mask(red_dims[e[1]],red_dims[e[0]],dims[e[1]],dims[e[0]])
            p -= lr * (mask_grad * p.grad)
    
    print(L)
    
    return model
    

In [223]:
lr = 0.001
x = {'a': torch.tensor([[1.0]]),'b': torch.tensor([[1.0,1.0]])}

gamma(quiverNN, x, lr);

tensor(1.3496, grad_fn=<AddBackward0>)


In [227]:
lr = 0.001
x = {'a': torch.tensor([[1.0]]),'b': torch.tensor([[1.0,1.0]])}

gammaPGD(quiverNN, x, lr);

{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}
{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}
{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}
{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}
{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}
{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}
{'bias': 1, 'a': 1, 'b': 2, 'c': 4, 'd': 2, 'e': 6}
tensor(7.5227, grad_fn=<AddBackward0>)


## Old Code

In [158]:
#################################
## Loss function (mean square error)
#################################

def loss_fn(y_predict, y_train):
    squared_diffs = (y_predict - y_train)**2
    return squared_diffs.mean()


#################################
## Masks used in projected gradient descent
#################################

def single_mask(red_rows, red_cols, orig_rows, orig_cols):
    assert red_rows <= orig_rows 
    assert red_cols <= orig_cols
    mask = torch.ones((orig_rows, orig_cols))
    for j in range(red_rows, orig_rows):
        mask[j][:red_cols] = 0
    return mask

def masks(reduced_dims, orig_dims):
    N = len(reduced_dims)
    assert N == len(orig_dims), \
    "Lengths must match"
    assert all([reduced_dims[i] <= orig_dims[i] for i in range(N)]), \
        "Reduced dimension vector must not be larger in any coordinate"
    
    masks = []
    
    # Add masks in order corresponding to the parameters
    for i in range(N-1):
        masks.append(single_mask(reduced_dims[i+1], reduced_dims[i], orig_dims[i+1], orig_dims[i]))
        masks.append(torch.transpose(single_mask(reduced_dims[i+1], 1, orig_dims[i+1],1 ), 0,1).reshape(orig_dims[i+1]))
    
    return(masks)


#################################
## Training loop with usual gradient descent
## Optimizer excluded to remove randomness
#################################

def training_loop(n_epochs, learning_rate, model, params, x_train, y_train, verbose=True):
    for epoch in range(1, n_epochs + 1):
        for p in params:
 
          if p.grad is not None: 
                p.grad.zero_()
        
        y_pred = model(x_train) 
        loss = loss_fn(y_pred, y_train)            
        loss.backward()
        
        with torch.no_grad(): 
            for p in params:
                p -= learning_rate * p.grad
                
        if verbose:
            if epoch ==1 or epoch % 500 == 0:
                print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return model


#################################
## Training loop with projected gradient descent
## Optimizer excluded to remove randomness
#################################

def training_loop_proj_GD(n_epochs, learning_rate, model, params, original_dimensions, reduced_dimensions, \
                          x_train, y_train, verbose=True):
    
    param_masks = masks(reduced_dimensions, original_dimensions)
    
    for epoch in range(1, n_epochs + 1):
        for p in params:
            if p.grad is not None: 
                p.grad.zero_()
        
        y_pred = model(x_train) 
        loss = loss_fn(y_pred, y_train)            
        loss.backward()
        
        with torch.no_grad(): 
            for p,m in zip(params, param_masks):
                assert p.shape == m.shape, "Parameter shape and mask shape don't match"
                
                # Use the mask to zero out gradients that will not be updated
                p -= learning_rate * (p.grad * m)
                
        if verbose:
            if epoch ==1 or epoch % 500 == 0:
                print('Epoch %d, Loss %f' % (epoch, float(loss)))
            
    return model

# Scraps

In [ ]:
class vertex:
    def __init__(self):
        None

In [ ]:
a = vertex()

In [ ]:
# Troubleshooting

W1 = np.random.rand(4, 2)
print(type(W1))
print(isinstance(W1, np.ndarray))

In [ ]:
a = np.random.rand(4,2)
b = np.random.rand(4,4)
a, b

In [ ]:
c = np.hstack((a,b))
np.shape(c)

In [ ]:
b

In [ ]:
b[:,:2]

In [ ]:
b[:,2:]

In [ ]:
np.shape(b[:,:2])

In [ ]:
np.shape(np.array([]))

In [186]:
torch.nn.Parameter(torch.tensor([[1.0]],dtype = torch.float,name="a to b"))

TypeError: tensor() got an unexpected keyword argument 'name'